## Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream [EEG+EDA+RESP_BELT] 

## Using Adaptive Random Forest 


- - New ensemble approach (Reward Penalty based Weighted Ensemble [RPWE])


## Date: 23 Sept 2021 at 10:10 AM

In [1]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from skmultiflow.meta import AdaptiveRandomForest
from window_slider import Slider

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):    
    list_values = list_values[0,:]
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [3]:
#======================================================
# EDA Feature Extraction (Wavelet Features)
#======================================================
def extract_eda_features(raw_eda):
    features =[]
    EDA = raw_eda
    list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features


In [4]:
#======================================================
# ECG Feature Extraction (Wavelet Features)
#======================================================

def extract_ecg_features(raw_data):
    ch =0
    features =[]
    
    def get_features(list_values):    
        list_values = list_values
        entropy = calculate_entropy(list_values)
        crossings = calculate_crossings(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + crossings + statistics

    for i in range(raw_data.shape[0]):
            ch_data = raw_data[i]
            list_coeff = pywt.wavedec(ch_data, 'db4', level=3)
            for coeff in list_coeff:
                features += get_features(coeff)
            ch = ch+1

    return features

In [5]:
def eeg_features(raw_data):
    ch = 0
    features= []
    def calculate_entropy(list_values):
        counter_values = Counter(list_values).most_common()
        probabilities = [elem[1]/len(list_values) for elem in counter_values]
        entropy=scipy.stats.entropy(probabilities)
        return entropy

    def calculate_statistics(list_values):
        median = np.nanpercentile(list_values, 50)
        mean = np.nanmean(list_values)
        std = np.nanstd(list_values)
        var = np.nanvar(list_values)
        rms = np.nanmean(np.sqrt(list_values**2))
        return [median, mean, std, var, rms]

    def get_features(list_values):    
    #     list_values = list_values[0,:]
        entropy = calculate_entropy(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + statistics
    
    for i in range(raw_data.shape[0]):
        ch_data = raw_data[i]
        list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
        for coeff in list_coeff:
            features += get_features(coeff)
            
        ch = ch+1
    return features

In [6]:
# ##===================================================
# # EEG data read from files
# ##===================================================
# def eeg_data(p,v):
#     file_eeg = '/home/gp/Desktop/MER_arin/data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
#     print(file_eeg)
#     df = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
#     eeg_sig = df.loc[df.iloc[:,1] == v]
#     return eeg_sig

# ##===================================================
# # EDA data read from files
# ##===================================================
# def eda_data(p,v):
#     file_eda = '/home/gp/Desktop/MER_arin/data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
#     print(file_eda)
#     df = pd.read_csv(file_eda,sep=',', header = None, engine='python')
#     eda_sig = df.loc[df.iloc[:,1] == v]
#     return eda_sig

# ##===================================================
# # Resp data read from files
# ##===================================================
# def resp_data(p,v):
#     file_resp = '/home/gp/Desktop/MER_arin/data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
#     print(file_resp)
#     df = pd.read_csv(file_resp,sep=',', header = None, engine='python')
#     resp_sig = df.loc[df.iloc[:,1] == v]
#     return resp_sig

In [8]:
#================================================================
# Initialization of different parameters and performance metrics
#================================================================
segment_in_sec = 10 #in sec
bucket_size = int((7085/51)*segment_in_sec)  #7085 is for 51 sec video record
overlap_count = 0

num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers

cBbest_val = [] #Classifiers least error for valence classsification
cBest_aro = []  #Classifiers least error for arousal classsification
cWorst_val = [] #Classifiers heighest error for valence classsification
cWorst_aro = []  #Classifiers heighest error for arousal classsification

val_MSE = []
aro_MSE = []

beta_val = np.ones(num_classifiers)/num_classifiers
beta_aro = np.ones(num_classifiers)/num_classifiers

n_epoch =1
epochs =  1 #epoch is 1 because the model will be trained only once

optimizer= 'ARF' #optimizer
classifier = 'ARF_'+str(optimizer)
indx = 0
run = 1

participant = 40
videos = 16

global eeg_emotion, eda_emotion, ecg_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
ecg_emotion = []
mer_emotion = []

all_eta =[]
init_m = 0

#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()

eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()

ecg_acc_val = metrics.Accuracy() #Accuracy
ecg_f1m_val = metrics.F1() #F1 measure  
ecg_acc_aro = metrics.Accuracy() #Accuracy
ecg_f1m_aro = metrics.F1() #F1 measure
ecg_roc_val = metrics.ROCAUC()
ecg_roc_aro = metrics.ROCAUC()
ecg_mcc_val = metrics.MCC()
ecg_mcc_aro = metrics.MCC()

 
mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
ecg_cm_val  = metrics.ConfusionMatrix()
ecg_cm_aro  = metrics.ConfusionMatrix()
    
itr = 0 #controls the learning rate


for ii in range(0,participant):

        p =ii+1 #participant number
        
        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
        ##===================================================
        # EEG data read from files
        ##===================================================
        def eeg_data(p):
            file_eeg = '/home/gp/Desktop/MER_arin/AMIGOS_data/eeg_data/'+str(p)+'data_eeg_AMIGOS'+'.csv'
            print(file_eeg)
            eeg_sig = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
            return eeg_sig

        ##===================================================
        # EDA data read from files
        ##===================================================
        def eda_data(p):
            file_eda = '/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/'+str(p)+'data_eda_AMIGOS.csv'
            print(file_eda)
            eda_sig = pd.read_csv(file_eda,sep=',', header = None, engine='python')
            return eda_sig

        ##===================================================
        # ECG data read from files
        ##===================================================
        def ecg_data(p):
            file_ecg = '/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/'+str(p)+'data_ecg_AMIGOS.csv'
            print(file_ecg)
            ecg_sig = pd.read_csv(file_ecg,sep=',', header = None, engine='python')
            return ecg_sig
        
        #------------------------------------
        # Once file fetched data stored here 
        #------------------------------------
        grand_eeg = eeg_data(p)
        grand_eda = eda_data(p)
        grand_ecg = ecg_data(p)

        
        for jj in range(0,videos):
            v = jj+1 #Video number
            print('=========================================================================')
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            print(p_v)
            
            emotion_label =[]
            
            
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = grand_eeg.loc[grand_eeg.iloc[:,1] == v]
            eda_sig = grand_eda.loc[grand_eda.iloc[:,1] == v]
            ecg_sig = grand_ecg.loc[grand_ecg.iloc[:,1] == v]
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            aro = eeg_sig.iloc[0,7087]
            val = eeg_sig.iloc[0,7088]
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al])
            
            #==========================================================
            # Predicted Valence and Arousal labels list initialization
            #==========================================================
            eeg_val_prdt=[]
            eeg_aro_prdt =[]
            
            eda_val_prdt=[]
            eda_aro_prdt =[]
            
            ecg_val_prdt=[]
            ecg_aro_prdt =[]
            
            
            #=========================================
            # Sliding window starts here 
            #=========================================
            slider_eeg = Slider(bucket_size,overlap_count)
            slider_eda = Slider(bucket_size,overlap_count)
            slider_ecg = Slider(bucket_size,overlap_count)
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,14),range(2,7087)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(2,7087)]) #keeping only eda signals
            ecg_sig = np.array(ecg_sig.iloc[range(0,2),range(2,7087)]) #keeping only ecg signals
            
            slider_eeg.fit(eeg_sig)
            slider_eda.fit(eda_sig)
            slider_ecg.fit(ecg_sig)

            while True:
                window_data_eeg = slider_eeg.slide()
                window_data_eda = slider_eda.slide() 
                window_data_ecg = slider_ecg.slide() 
                
                #=================================================
                # Feature extraction from EEG
                #=================================================
                window_data_eeg  = np.nan_to_num(window_data_eeg) #Replacing NaN with 0 if there is such case
                features_eeg = eeg_features(window_data_eeg)
                x_eeg = np.array([features_eeg])  #EEG raw feature vector
                
#                 x_eeg = preprocessing.normalize(eeg) # EEG normalized features [0,1] 
                
                
                #=================================================
                # Feature extraction from EDA
                #=================================================
                window_data_eda  = np.nan_to_num(window_data_eda) #Replacing NaN with 0 if there is such case
                eda_features = extract_eda_features(np.array(window_data_eda))
                x_eda = np.array([eda_features]) #EDA raw feature vector
#                 x_eda = preprocessing.normalize(eda) #EDA normalized features
                
                #=================================================
                # Feature extraction from ECG
                #=================================================
                window_data_ecg  = np.nan_to_num(window_data_ecg) #Replacing NaN with 0 if there is such case
                ecg_features = extract_ecg_features(np.array(window_data_ecg))
                x_ecg = np.array([ecg_features]) #ECG raw feature vector
#                 x_ecg = preprocessing.normalize(ecg) #ECG normalized features
            
            
                #===================================================
                # Model initialization
                #===================================================
                if init_m == 0:
                    print('EEG Feature shape{}:'.format(x_eeg.shape))
                    print('EDA Feature shape{}:'.format(x_eda.shape))
                    print('ecg BELT Feature shape{}:'.format(x_ecg.shape))

                    #========================
                    # For EEG data MLP model
                    #========================
                    eeg_model_val = AdaptiveRandomForest()
                    eeg_model_aro = AdaptiveRandomForest()

                    #========================
                    # For EDA data MLp model
                    #========================
                    eda_model_val = AdaptiveRandomForest()
                    eda_model_aro = AdaptiveRandomForest()
                    

                    #==============================
                    # For ecg Belt data MLP Model
                    #==============================
                    ecg_model_val = AdaptiveRandomForest()
                    ecg_model_aro = AdaptiveRandomForest()


                    init_m = init_m+1


                #===============================================================
                # Emotion Classification --> Valence and Arousal
                #===============================================================

                #===========================================
                # From EEG data -- RECS System
                #===========================================

                #Valence classification EEG

                #Test the model first 
                if (indx == 0): #When the model sees the first data
                    tmp_eeg_val = 0
                    y_pred_val_eeg = 2 #other than what are class labels
                else: 
                    eeg_val = eeg_model_val.predict_proba(x_eeg)
                    tmp_eeg_val = np.max(eeg_val)
                    y_pred_val_eeg = np.argmax(eeg_val)
                    
                #Train the model once
                eeg_model_val.partial_fit(x_eeg,y_act_val)

                eeg_acc_val = eeg_acc_val.update(y_act_val[0], y_pred_val_eeg)  # update the accuracy metric

                eeg_f1m_val = eeg_f1m_val.update(y_act_val[0], y_pred_val_eeg) #update f1 measure metric
                
                eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg)
                
                eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg)

                #Arousal classification EEG

                if (indx == 0): #When the model sees the first data
                    tmp_eeg_aro = 0
                    y_pred_aro_eeg = 2 #other than what are class labels
                else: 
                    eeg_aro = eeg_model_aro.predict_proba(x_eeg)
                    tmp_eeg_aro = np.max(eeg_aro)
                    y_pred_aro_eeg = np.argmax( eeg_aro)

                #Train the model once
                eeg_model_aro.partial_fit(x_eeg,y_act_aro)

                eeg_acc_aro = eeg_acc_aro.update(y_act_aro[0], y_pred_aro_eeg)  # update the accuracy metric

                eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro[0], y_pred_aro_eeg) #update f1 measure metric
                
                eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg)
                
                eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg)

                #===========================================
                # From EDA data 
                #===========================================

                #Valence classification EDA

                if (indx == 0): #When the model sees the first data
                    tmp_eda_val = 0
                    y_pred_val_eda = 2 #other than what are class labels
                else: 
                    eda_val = eda_model_val.predict_proba(x_eda) 
                    tmp_eda_val = np.max(eda_val)
                    y_pred_val_eda = np.argmax( eda_val)

                #Train the model once
                eda_model_val.partial_fit(x_eda,y_act_val)

                eda_acc_val = eda_acc_val.update(y_act_val[0], y_pred_val_eda)  # update the accuracy metric

                eda_f1m_val = eda_f1m_val.update(y_act_val[0], y_pred_val_eda) #update f1 measure metric
                
                eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda)
                
                eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda)
                

                #Arousal classification EDA
                if (indx == 0): #When the model sees the first data
                    tmp_eda_aro = 0
                    y_pred_aro_eda = 2 #other than what are class labels
                else: 
                    eda_aro = eda_model_aro.predict_proba(x_eda)
                    tmp_eda_aro = np.max(eda_aro)
                    y_pred_aro_eda = np.argmax(eda_aro)

                #Train the model once
                eda_model_aro.partial_fit(x_eda,y_act_aro)

                eda_acc_aro = eda_acc_aro.update(y_act_aro[0], y_pred_aro_eda)  # update the accuracy metric

                eda_f1m_aro = eda_f1m_aro.update(y_act_aro[0], y_pred_aro_eda) #update f1 measure metric
                
                eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda)
                
                eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda)

                #===========================================
                # From ecg Belt data
                #===========================================

                #Valence classification ecg Belt

                if (indx == 0): #When the model sees the first data
                    tmp_ecg_val = 0
                    y_pred_val_ecg = 2 #other than what are class labels
                else: 
                    ecg_val = ecg_model_val.predict_proba(x_ecg)
                    tmp_ecg_val = np.max(ecg_val)
                    y_pred_val_ecg = np.argmax(ecg_val)

                #Train the model once
                ecg_model_val.partial_fit(x_ecg,y_act_val)

                ecg_acc_val = ecg_acc_val.update(y_act_val[0], y_pred_val_ecg)  # update the accuracy metric

                ecg_f1m_val = ecg_f1m_val.update(y_act_val[0], y_pred_val_ecg) #update f1 measure metric
                
                ecg_roc_val = ecg_roc_val.update(y_act_val[0], y_pred_val_ecg)
                
                ecg_mcc_val = ecg_roc_val.update(y_act_val[0], y_pred_val_ecg)
                

                #Arousal classification ecg Belt
                if (indx == 0): #When the model sees the first data
                    tmp_ecg_aro = 0
                    y_pred_aro_ecg = 2 #other than what are class labels
                else: 
                    ecg_aro = ecg_model_aro.predict_proba(x_ecg)
                    tmp_ecg_aro = np.max(ecg_aro)
                    y_pred_aro_ecg = np.argmax(ecg_aro)
               

                #Train the model once
                ecg_model_aro.partial_fit(x_ecg,y_act_aro)

                ecg_acc_aro = ecg_acc_aro.update(y_act_aro[0], y_pred_aro_ecg)  # update the accuracy metric

                ecg_f1m_aro = ecg_f1m_aro.update(y_act_aro[0], y_pred_aro_ecg) #update f1 measure metric
                
                ecg_roc_aro = ecg_roc_aro.update(y_act_aro[0], y_pred_aro_ecg)
                
                ecg_mcc_aro = ecg_mcc_aro.update(y_act_aro[0], y_pred_aro_ecg)
                
                
                indx = indx+1

                if slider_eeg.reached_end_of_list(): break
                
            #===================================================
            #   Error storing and comparing Part of each models
            #===================================================
            
            eeg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eeg_val))
            eeg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eeg_aro))
            
            eda_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eda_val))
            eda_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eda_aro))
            
            ecg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_ecg_val))
            ecg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_ecg_aro))
            
            val_MSE = [eeg_val_MSE, eda_val_MSE, ecg_val_MSE] #Valence errors 
            aro_MSE = [eeg_aro_MSE, eda_aro_MSE, ecg_aro_MSE] #Arousal errors
            

            
            #========================================================
            # Valence Arousal MSE Compariosn and storing in Cbest and Cworst
            #========================================================
            
            if (jj==0): #First Video
                cBest_val = val_MSE
                cBest_aro = aro_MSE
                
                cWorst_val  = val_MSE
                cWorst_aro = aro_MSE
                
            else:
                #-----------------------------------------
                #Beta for calence and arousal calculation
                #-----------------------------------------              
                beta_val = np.true_divide(list(np.subtract(cBest_val,val_MSE)), (1+np.exp(list(np.subtract(cWorst_val,cBest_val)))))
                beta_aro = np.true_divide(list(np.subtract(cBest_aro,aro_MSE)), (1+np.exp(list(np.subtract(cWorst_aro,cBest_aro)))))
                
                #--------------------------------------------------------------
                # Classifiers best MSE error and worst MSE error value update
                #-------------------------------------------------------------
                cBest_val = np.minimum(cBest_val, val_MSE)
                cBest_aro = np.minimum(cBest_aro, aro_MSE)
                cWorst_val = np.maximum(cWorst_val, val_MSE)
                cWorst_aro = np.maximum(cWorst_aro, aro_MSE)
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg)
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg)
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda)
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda)
            
            ecg_cm_val = ecg_cm_val.update(y_act_val[0], y_pred_val_ecg)
            ecg_cm_varo = ecg_cm_aro.update(y_act_aro[0], y_pred_aro_ecg)            
            
            #=============================================================
            # Storing All Results
            #=============================================================
            
            emotion_label.append([y_pred_val_eeg, y_pred_aro_eeg]) #appending valence & arousal predicted EEG
            emotion_label.append([y_pred_val_eda, y_pred_aro_eda]) #appending valence & arousal predicted EDA
            emotion_label.append([y_pred_val_ecg, y_pred_aro_ecg]) #appending valence & arousal predicted ecg

            emotion_label = np.array(emotion_label)
            
                    
            #====================================================================
            # Decision label ensemble --> Reward Penalty Based Weightrd Ensemble
            #====================================================================
            val_label = emotion_label[:,0]
            aro_label = emotion_label[:,1]
            
            #------------------------------------------
            # Valence Class ensemble
            #------------------------------------------
            
            p_val = np.dot([w_val],val_label)
            y_prdt_mer_val = (p_val -0.5)
            if(y_prdt_mer_val > 0):
                mer_val = 1
            else:
                mer_val = 0

            #============================================
            # Weight update for Valence
            #============================================
            w_val = w_val + list(np.multiply(w_val,beta_val))    
            w_val_sum = sum(w_val) #total sum of weights
            w_val = np.array(w_val/w_val_sum) #weight rescaling


            
            #------------------------------------------
            # Arousal Class ensemble
            #------------------------------------------            
            p_val = np.dot([w_aro],aro_label)
            y_prdt_mer_aro = (p_val-0.5)
            if(y_prdt_mer_aro > 0):
                mer_aro = 1
            else:
                mer_aro = 0

            w_aro = w_aro + list(np.multiply(w_aro,beta_aro))
            w_aro_sum = sum(w_aro) #total sum of weights
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro = mer_cm_aro.update(y_act_aro[0], mer_aro)
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                         eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                         y_act_val[0], y_pred_val_eeg, y_act_aro[0], y_pred_aro_eeg]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                         eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                         y_act_val[0], y_pred_val_eda, y_act_aro[0], y_pred_aro_eda]))
            
            ecg_emotion.append(np.array([p,v, ecg_acc_val.get(), ecg_f1m_val.get(), ecg_roc_val.get(), ecg_mcc_val.get(),
                                          ecg_acc_aro.get(),ecg_f1m_aro.get(), ecg_roc_aro.get(), ecg_mcc_aro.get(), 
                                          y_act_val[0], y_pred_val_ecg, y_act_aro[0], y_pred_aro_ecg]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                         mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                         y_act_val[0], mer_val, y_act_aro[0], mer_aro]))
            print('Actual emotion [valence: {}, arousal: {}]'.format(y_act_val, y_act_aro))
            print('Single modals Predicted emotion')
            print('[valence, arousal]:')
            print(emotion_label)
            
            print('MER Predicted emotion [valence: {}, arousal: {}]'.format(mer_val, mer_aro))
#             print('Val MSE',val_MSE)
#             print('Aro MSE',aro_MSE)
#             # print('------------------------------')
#             print('Beta_Val:', beta_val)
#             print('Beta_Aro:', beta_aro)
#             print('------------------------------')
#             print('W Val:', w_val)
#             print('W Aro:', w_aro)
            # print('------------------------------')
            # print('CBest Val MSE',cBest_val)
            # print('CBest Aro MSE',cBest_aro)
            # print('------------------------------')
            # print('Cworst Val MSE',cWorst_val)
            # print('Cworst Aro MSE',cWorst_aro)
            # print('==============================================')            
            # print('..................................................')
            # print('EEG')

            # print("Valence Accuracy:{}".format(round(eeg_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eeg_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eeg_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eeg_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eeg_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eeg_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eeg_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eeg_mcc_aro.get(),4)))

            # print('..................................................')
            # print('EDA')

            # print("Valence Accuracy:{}".format(round(eda_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eda_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eda_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eda_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eda_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eda_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eda_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eda_mcc_aro.get(),4)))


            # print('..................................................')
            # print('ecg BELT')

            # print("Valence Accuracy:{}".format(round(ecg_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(ecg_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(ecg_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(ecg_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(ecg_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(ecg_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(ecg_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(ecg_mcc_aro.get(),4)))

            # print('..................................................')
            print('MER model')

            print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            print('===============================xxxx================================')
                
                
                
            



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/AMIGOS_data/eeg_data/1data_eeg_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/1data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/1data_ecg_AMIGOS.csv
Person: 1 Video:1
EEG Feature shape(1, 504):
EDA Feature shape(1, 48):
ecg BELT Feature shape(1, 96):
Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:1.0
Valence F1 score:0.0
Valence ROC score:-0.0
Valence MCC score:0.0
Arousal Accuracy:1.0
Arousal F1 score:0.0
Arousal ROC score:-0.0
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:2
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, ar

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [1 1]
 [0 1]]
MER Predicted emotion [valence: 1, arousal: 1]
MER model
Valence Accuracy:0.6875
Valence F1 score:0.6154
Valence ROC score:0.6875
Valence MCC score:0.4045
Arousal Accuracy:0.75
Arousal F1 score:0.6
Arousal ROC score:0.7
Arousal MCC score:0.4472
===============================xxxx================================
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/AMIGOS_data/eeg_data/2data_eeg_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/2data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/2data_ecg_AMIGOS.csv
Person: 2 Video:1
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [1 0]
 [1 0]]
MER Predicted emotion [valence: 1, arousal: 0]
MER model
Valence Accuracy:0.7059
Valence F1 score:0.6667
Valence ROC score:0.

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.6452
Valence F1 score:0.56
Valence ROC score:0.6324
Valence MCC score:0.2753
Arousal Accuracy:0.6774
Arousal F1 score:0.375
Arousal ROC score:0.5864
Arousal MCC score:0.2247
===============================xxxx================================
Person: 2 Video:16
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [1 0]
 [0 0]]
MER Predicted emotion [valence: 1, arousal: 0]
MER model
Valence Accuracy:0.6563
Valence F1 score:0.5926
Valence ROC score:0.649
Valence MCC score:0.3072
Arousal Accuracy:0.6875
Arousal F1 score:0.375
Arousal ROC score:0.5887
Arousal MCC score:0.2322
===============================xxxx================================
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/hom

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.6739
Valence F1 score:0.5946
Valence ROC score:0.6596
Valence MCC score:0.3279
Arousal Accuracy:0.6522
Arousal F1 score:0.3333
Arousal ROC score:0.5583
Arousal MCC score:0.1466
===============================xxxx================================
Person: 3 Video:15
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [1 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.6596
Valence F1 score:0.5789
Valence ROC score:0.6465
Valence MCC score:0.3032
Arousal Accuracy:0.6383
Arousal F1 score:0.32
Arousal ROC score:0.551
Arousal MCC score:0.1304
===============================xxxx================================
Person: 3 Video:16
Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, a

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.623
Valence F1 score:0.5306
Valence ROC score:0.6084
Valence MCC score:0.2242
Arousal Accuracy:0.623
Arousal F1 score:0.439
Arousal ROC score:0.5931
Arousal MCC score:0.22
===============================xxxx================================
Person: 4 Video:14
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [0 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
MER model
Valence Accuracy:0.629
Valence F1 score:0.549
Valence ROC score:0.6176
Valence MCC score:0.2424
Arousal Accuracy:0.6129
Arousal F1 score:0.4286
Arousal ROC score:0.581
Arousal MCC score:0.1874
===============================xxxx================================
Person: 4 Video:15
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousa

...

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5881
Valence F1 score:0.5597
Valence ROC score:0.5866
Valence MCC score:0.1737
Arousal Accuracy:0.6911
Arousal F1 score:0.7122
Arousal ROC score:0.6892
Arousal MCC score:0.3791
===============================xxxx================================
Person: 36 Video:14
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5871
Valence F1 score:0.5587
Valence ROC score:0.5856
Valence MCC score:0.1718
Arousal Accuracy:0.6916
Arousal F1 score:0.7122
Arousal ROC score:0.6898
Arousal MCC score:0.3803
===============================xxxx================================
Person: 36 Video:15
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valen

Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5867
Valence F1 score:0.5525
Valence ROC score:0.5845
Valence MCC score:0.1698
Arousal Accuracy:0.6854
Arousal F1 score:0.7049
Arousal ROC score:0.6838
Arousal MCC score:0.3685
===============================xxxx================================
Person: 37 Video:13
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5857
Valence F1 score:0.5515
Valence ROC score:0.5835
Valence MCC score:0.168
Arousal Accuracy:0.6859
Arousal F1 score:0.7059
Arousal ROC score:0.6842
Arousal MCC score:0.3694
===============================xxxx================================
Person: 37 Video:14
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valenc

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5871
Valence F1 score:0.5481
Valence ROC score:0.5843
Valence MCC score:0.1697
Arousal Accuracy:0.6833
Arousal F1 score:0.702
Arousal ROC score:0.6818
Arousal MCC score:0.3643
===============================xxxx================================
Person: 38 Video:12
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5861
Valence F1 score:0.5471
Valence ROC score:0.5834
Valence MCC score:0.1679
Arousal Accuracy:0.6838
Arousal F1 score:0.703
Arousal ROC score:0.6823
Arousal MCC score:0.3652
===============================xxxx================================
Person: 38 Video:13
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 0]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
MER model
Valence Accuracy:0.5809
Valence F1 score:0.5416
Valence ROC score:0.5782
Valence MCC score:0.1575
Arousal Accuracy:0.6796
Arousal F1 score:0.6972
Arousal ROC score:0.6784
Arousal MCC score:0.3573
===============================xxxx================================
Person: 39 Video:11
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [0 1]
 [1 0]]
MER Predicted emotion [valence: 1, arousal: 0]
MER model
Valence Accuracy:0.58
Valence F1 score:0.5406
Valence ROC score:0.5772
Valence MCC score:0.1555
Arousal Accuracy:0.6785
Arousal F1 score:0.6962
Arousal ROC score:0.6773
Arousal MCC score:0.355
===============================xxxx================================
Person: 39 Video:12
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence,

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 0]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5782
Valence F1 score:0.5373
Valence ROC score:0.5755
Valence MCC score:0.1522
Arousal Accuracy:0.6746
Arousal F1 score:0.6907
Arousal ROC score:0.6736
Arousal MCC score:0.3474
===============================xxxx================================
Person: 40 Video:10
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5789
Valence F1 score:0.5373
Valence ROC score:0.576
Valence MCC score:0.1533
Arousal Accuracy:0.6735
Arousal F1 score:0.6897
Arousal ROC score:0.6725
Arousal MCC score:0.3453
===============================xxxx================================
Person: 40 Video:11
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valenc

In [9]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('ecg Val CM:')
print(ecg_cm_val)
print('ecg Aro CM:')
print(ecg_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    217    115
      1    149    159
EEG aro CM:
             0      1
      0    210     93
      1    108    229
EDA Val CM:
             0      1
      0    237     95
      1    128    180
EDA aro CM:
             0      1
      0    220     83
      1    100    237
ecg Val CM:
             0      1
      0    204    128
      1    148    160
ecg Aro CM:
             0      1
      0    193    110
      1    104    233
ReMECS Val CM:
             0      1
      0    213    119
      1    151    157
ReMECS aro CM:
             0      1
      0    197    106
      1    104    233


In [10]:
print('[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]')
#=============================
# EEG
#=============================
t_eeg_emotion =pd.DataFrame(eeg_emotion) 
t_eeg_emotion = t_eeg_emotion.iloc[:,range(2,10)]
t_eeg_emotion_avg = np.array(np.mean(t_eeg_emotion))
t_eeg_emotion_std = np.array(np.std(t_eeg_emotion))

print('EEG result Average:', t_eeg_emotion_avg)
print('EEG result std:', t_eeg_emotion_std)

#=============================
# EDA
#=============================
t_eda_emotion =pd.DataFrame(eda_emotion) 
t_eda_emotion = t_eda_emotion.iloc[:,range(2,10)]
t_eda_emotion_avg = np.array(np.mean(t_eda_emotion))
t_eda_emotion_std = np.array(np.std(t_eda_emotion))

print('EDA result Average:', t_eda_emotion_avg)
print('EDA result std:', t_eda_emotion_std)
#=============================
# ecg BELT
#=============================
t_ecg_emotion =pd.DataFrame(ecg_emotion) 
t_ecg_emotion = t_ecg_emotion.iloc[:,range(2,10)]
t_ecg_emotion_avg = np.array(np.mean(t_ecg_emotion))
t_ecg_emotion_std = np.array(np.std(t_ecg_emotion))

print('ecg Belt result Average:', t_ecg_emotion_avg)
print('ecg Belt result std:', t_ecg_emotion_std)
#=============================
# MER
#=============================
t_mer_emotion =pd.DataFrame(mer_emotion) 
t_mer_emotion = t_mer_emotion.iloc[:,range(2,10)]
t_mer_emotion_avg = np.array(np.mean(t_mer_emotion))
t_mer_emotion_std = np.array(np.std(t_mer_emotion))

print('MER result Average:', t_mer_emotion_avg)
print('MER result std:', t_mer_emotion_std)

[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]
EEG result Average: [0.65022682 0.62005865 0.6448306  0.29446864 0.72402631 0.68530782
 0.70727367 0.42969901]
EEG result std: [0.03082798 0.09299794 0.04676961 0.08474005 0.0292096  0.15845309
 0.07818352 0.09531009]
EDA result Average: [0.68119159 0.64261472 0.67484051 0.35943962 0.73716295 0.72067674
 0.72681759 0.46571165]
EDA result std: [0.02335543 0.07218409 0.04100779 0.0577219  0.02406366 0.08167103
 0.06313044 0.0611489 ]
ecg Belt result Average: [0.63684172 0.61475021 0.63236975 0.63236975 0.72151302 0.69806016
 0.70842114 0.42994561]
ecg Belt result std: [0.0238059  0.06510006 0.03768893 0.03768893 0.02730247 0.10418476
 0.06777633 0.07659592]
MER result Average: [0.61451931 0.58571598 0.60985508 0.22279265 0.70150841 0.67384807
 0.68588072 0.38444604]
MER result std: [0.03896189 0.05813177 0.04217288 0.07095351 0.03501636 0.11625326
 0.06676249 0.07828497]


In [11]:
time_stamp = datetime.datetime.now()
run =1
algo= 'ARF/' 
filename = "/home/gp/Desktop/MER_arin/mer_codes/11AMIGOS_RPWE/Results_RPWE_AMIGOS/"+algo
fname_eeg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EDA'+'_' +classifier+'_results.csv'
fname_ecg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_ecg_BELT'+'_' +classifier+'_results.csv'
fname_mer = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
ecg_emotion = pd.DataFrame(ecg_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
ecg_emotion.to_csv(fname_ecg)
mer_emotion.to_csv(fname_mer)


